This notebook includes the data manipulation used to bring the data to the desired form. We would like to define a feature vector for each tournament game played after 2003 through 2018 (2003 and 2008 included). To this end we define a data frame where each row corresponds to a team for a given year which has the team id, the year team played with the following statistics: 
The number of the games the team won,
the number of the games the team lost,
average score per game,
average field goal per game,
the number of field goal attemp per game,
the average 3 point scores per game,
the number of 3 point attemps per game,
the number of offensive rebounds per game,
the number of defensive rebounds per game,
the number of assists per game,
the number of steals per game,
the number of blocks per game,
the number of personal fouls per game,
average of the ranking of the team at the last day of the season,
the ratio of wins at home games,
the ratio of loose at home games,
the ratio of win at away games,
the ratio of loose at away games.
If a team joined more than one season than the team will have a row for each year joined to the season.

Then for a given game from the tournament at a particular year, we set either winner or loser team as Team 1 and the other one Team 2 randomly, substract the stats (from the regular season of the given year) of team 1 from team 2(team2-team1), set the difference as the feature vector, and if the team 1 wins we label the game 0 and if team 2 wins we label the game 1.

Lastly, to be able to use this data more than once we record all the data to a csv file.

Also, we repeat the same for the tournament games at 2019.

Main objective is to train our model using the games from 2003 through 2018 and predict the results of the games of 2019 using the regular season features.

In [2]:
import os
import numpy as np
import pandas as pd
import pixiedust
import sklearn as sk
import csv


Pixiedust database opened successfully


We first import the necessary packages and the data. Data files should be placed in a folder name mm_data located at the same place with notebook.

In [3]:
all_teams = pd.read_csv("mm_data\MDataFiles_Stage1\MTeams.csv")
reg_se_dt = pd.read_csv("mm_data\MDataFiles_Stage1\MRegularSeasonDetailedResults.csv")
tour_com = pd.read_csv("mm_data\MDataFiles_Stage1\MNCAATourneyCompactResults.csv")
rankings = pd.read_csv("mm_data\MDataFiles_Stage1\MMasseyOrdinals.csv")

In [4]:
reg_se_cm = pd.read_csv("mm_data\MDataFiles_Stage1\MRegularSeasonCompactResults.csv")
reg_se_cm = reg_se_cm.query('Season >= 2003')

The following function is used to calculate home-away statistics of a team.

In [5]:
def home_away(team_id,year,data):
    #this function takes inputs a team id(int), a particular year(int) and a data(data frame). It returns the ratio of
    #home wins to home games,and away wins to away games.
    data_year = data.loc[data['Season'] == year]
    #seperate the data of given year
    team_wins_home = len(data_year[(data_year['WTeamID'] == team_id) & (data_year['WLoc'] =='H')])
    team_los_home = len(data_year[(data_year['LTeamID'] == team_id) & (data_year['WLoc'] =='H')])
    #count home wins and home loose
    team_wins_away = len(data_year[(data_year['WTeamID'] == team_id) & (data_year['WLoc'] =='A')])
    team_los_away = len(data_year[(data_year['LTeamID'] == team_id) & (data_year['WLoc'] =='A')])
    #count home wins and home loose
    #return the ratios
    return team_wins_home/(team_wins_home+team_los_home), team_wins_away/(team_los_away+team_wins_away)

The following function is used to find the rank average statistics of a given team.

In [6]:
def team_rank(year, teamid, data):
    #this function takes input of year number, day number, ranking data and returns the ranking of the team at given year given date.
    day_num =  data.loc[(data['Season'] == year) & (data['TeamID'] == teamid)].RankingDayNum.max()
    #determine the last day which ranking information is given.
    ranks = data.loc[(data['Season'] == year) & (data['TeamID'] == teamid) & (data['RankingDayNum'] == day_num), 'OrdinalRank']
    #take the average of rankings at the given year in the last day and return the average for the given team.
    return ranks.mean()

The function defined below will be used in the next block to turn the detailed game data of seasons 2003 - 2018 into a dictionary. The function itself takes team id (id conventions are coming from the data set) and the dataset and returns several stats of given team as a data frame.

In [23]:
def team_data_col(teamid, datas):
    #this function takes an integer input which stands for team id and exports the data information of the given team.  
    team_data = {}
    #create a dictionary to keep the statistics.
    team_winning_data = datas[datas.WTeamID == teamid]
    #take all the rows where the given team is winning team.
    team_losing_data = datas[datas.WTeamID == teamid]
    #take all the rows where the given team is losing team.
    #Use team_winning_data and team_losing_data to count the stats of the team since every game a team played it is either 
    #the winner of the loser.
    team_data['win'] = team_winning_data.shape[0]
    team_data['los'] = team_winning_data.shape[0]
    team_data['played'] = team_data['los'] + team_data['win']
    n = team_data.get('played')
    team_data['score_avg'] = (team_losing_data.LScore.sum() + team_winning_data.WScore.sum())/n
    team_data['fgoal_avg'] = (team_winning_data.WFGM.sum() + team_losing_data.LFGM.sum())/n
    team_data['fgoal_at_avg'] = (team_winning_data.WFGA.sum() + team_losing_data.LFGA.sum())/n
    team_data['pointer_avg'] = (team_winning_data.WFGM3.sum() + team_losing_data.LFGM3.sum())/n
    team_data['pointer_at_avg'] = (team_winning_data.WFGA3.sum() + team_losing_data.LFGA3.sum())/n
    team_data['off_reb_avg'] = (team_winning_data.WOR.sum() + team_losing_data.LOR.sum())/n
    team_data['def_reb_avg'] = (team_winning_data.WDR.sum() + team_losing_data.LDR.sum())/n
    team_data['ast_avg'] = (team_winning_data.WAst.sum() + team_losing_data.LAst.sum())/n
    team_data['turn_avg'] = (team_winning_data.WTO.sum() + team_losing_data.LTO.sum())/n
    team_data['stl_avg'] = (team_winning_data.WStl.sum() + team_losing_data.LStl.sum())/n
    team_data['block_avg'] = (team_winning_data.WBlk.sum() + team_losing_data.LBlk.sum())/n
    team_data['foul_avg'] = (team_winning_data.WPF.sum() + team_losing_data.LPF.sum())/n
    return team_data

In [26]:
def games_data_col(years, games_data, source_data):
    games_df = pd.DataFrame(columns=['game_id','result', 'win_diff','loss_diff', 'played_diff', 'score_av_diff','fgoal_diff', 'fgoal_at_diff', 'pointer_diff','pointer_at_diff', 'off_reb_diff', 'def_reb_diff','ast_diff', 'turn_diff', 'stl_diff','block_diff', 'foul_diff','rank1','rank2','home1', 'away1','home2', 'away2']) 
    active_games = games_data[games_data['Season'].isin(years)]
    for i in range(len(active_games)):
        #record the ids of winning and losing team.
        #record the year of the game
        year = active_games.Season.iloc[i]
        win_id = active_games.WTeamID.iloc[i]
        los_id = active_games.LTeamID.iloc[i]
        active_source = source_data[source_data['Season'] == year]
        #recover the data of winning and losing teams
        win_data = team_data_col(win_id, active_source)
        loss_data = team_data_col(los_id, active_source)
        #decide which team will be called 1. If dice gives 0 then Team 1 is winning and Team 2 is losing.
        #if dice gives 1 then Team 2 is winning and Team1 is winning
        dice = np.round_(np.random.random(),decimals = 0)
        #create data array for the game.
        if dice == 0:
            game_ide = str(win_id) +' vs '+ str(los_id)
            data_array = [game_ide,0]
            for key in win_data.keys():
                data_array.append(loss_data[key]-win_data[key])
            #add the difference of features of winning team and losing team to the data_array. Follow the convention Team2-Team1
            data_array.append(team_rank(year,win_id,rankings))
            data_array.append(team_rank(year,los_id,rankings))
            win_home, win_away =  home_away(win_id,year,reg_se_cm)
            los_home, los_away = home_away(los_id,year,reg_se_cm)
            data_array = data_array + [win_home,win_away,los_home,los_away]            
        elif dice == 1:
            game_ide = str(los_id) + ' vs ' + str(win_id)
            data_array = [game_ide,1]
            for key in win_data.keys():
                data_array.append(win_data[key]-loss_data[key])
            data_array.append(team_rank(year,los_id,rankings))
            data_array.append(team_rank(year,win_id,rankings))
            win_home, win_away =  home_away(win_id,year,reg_se_cm)
            los_home, los_away = home_away(los_id,year,reg_se_cm)
            data_array = data_array + [los_home,los_away,win_home,win_away]
        games_df.loc[len(games_df)] = data_array
    return games_df

In [25]:
games_data = games_data_col(range(2003,2018),tour_com,reg_se_dt)
games_data.to_csv('game_data_labels318.csv',index = False)

In [1]:
with open('game_data_labels318.csv', 'r') as t1, open('game_data_w_labels318.csv', 'r') as t2:
    fileone = t1.readlines()
    filetwo = t2.readlines()

with open('update.csv', 'w') as outFile:
    for line in filetwo:
        if line not in fileone:
            outFile.write(line)
t1.close()
t2.close()